In [42]:
%load_ext autoreload
%aimport data_creater
%aimport model
%autoreload 1

from data_creater import *
from model import *
import re, copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()
print(symbols)

['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'XOM', 'WBA', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS', 'GOOG', 'FB']


In [13]:
import gc 
# del df
# del stocks, Sequential
gc.collect()

199

In [63]:
other_symbols = [symbol_tem for symbol_tem in symbols if symbol_tem != symbol]
symbol = 'AAPL'
df = pd.read_csv('./data/{0}/all_normalized.csv'.format(symbol), index_col=[0], parse_dates=[0])
symbol_columns = [symbol_tem for symbol_tem in list(df.columns.values) if bool(re.match('normal.*', symbol_tem))]

box = []
for symbol_tem in other_symbols:
    box_tem = []
    for col in list(df.columns.values):
        if bool(re.match(symbol_tem + '_normal_.*', col)):
            box_tem.append(col)
    box.append(box_tem)

all_combination = []
all_combination.append(symbol_columns)
for box_tem in box:
    all_combination.append(symbol_columns + box_tem)
    
all_combination[0]

['normal_open_',
 'normal_high',
 'normal_low',
 'normal_volume',
 'normal_close',
 'normal_close_delta_1',
 'normal_close_delta_2',
 'normal_close_delta_3',
 'normal_returns',
 'normal_mfi']

In [77]:
ticker = 'AAPL'
window_size = 7
seq_obj = MultiSequence(ticker,window_size,2,False,all_combination[0])


# sys.exit()
X_train,y_train,X_test,y_test = split_data(seq_obj)
print(X_train[0].shape)
print(y_train[0].shape)
print(X_train[0])
print(y_train[0])
print('\n')
print(seq_obj.Xpred.shape)
print(y_train[0])

# seq_obj = MultiSequence(ticker,window_size,2,False)
# X_train,y_train,X_test,y_test = split_data(seq_obj)
# print(y_train[0])

# seq_obj = MultiSequence(ticker,window_size,3,False)
# X_train,y_train,X_test,y_test = split_data(seq_obj)
# print(y_train[0])

# seq_obj = MultiSequence(ticker,window_size,4,False)
# X_train,y_train,X_test,y_test = split_data(seq_obj)
# print(y_train[0])

# seq_obj = MultiSequence(ticker,window_size,5,False)
# X_train,y_train,X_test,y_test = split_data(seq_obj)
# print(y_train[0])

         date        open        high         low    volume       close  \
0  2019-01-25  155.479996  158.130005  154.320007  33535500  155.424744   
1  2019-01-28  155.789993  156.330002  153.660004  26192100  153.986359   
2  2019-01-29  156.250000  158.130005  154.110001  41587200  152.390320   
3  2019-01-30  163.250000  166.149994  160.229996  61109800  162.803864   
4  2019-01-31  166.110001  169.000000  164.559998  40739600  163.976242   

   close_delta_1  close_delta_2  close_delta_3   returns  ...  \
0       4.985108       3.783158       4.393967  0.032600  ...   
1      -1.438385       3.546723       2.344773 -0.009298  ...   
2      -1.596039      -3.034424       1.950684 -0.010419  ...   
3      10.413544       8.817505       7.379120  0.066101  ...   
4       1.172378      11.585922       9.989883  0.007175  ...   

   FB_normal_open_  FB_normal_high  FB_normal_low  FB_normal_volume  \
0        -0.908476       -0.891032      -0.867283         -0.566233   
1        -0.8901

In [71]:
window_sizes = [1,2,3,4]
dropouts =  [0.25,0.4]
learn_rates = [0.001]
epochs = [500,700]
batch_size = 30

In [78]:
result = model_selector('AAPL', window_sizes, learn_rates, dropouts, epochs, batch_size,target_length=2,all_day=False,verbose=1,column=all_combination[0])

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)


*** Best Model Selection for AAPL ***

Window size: 1
------------------------------------------------------------
1    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 500 Training error: 0.0048 Testing error: 0.0149
2    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 700 Training error: 0.0049 Testing error: 0.0129
3    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 500 Training error: 0.0052 Testing error: 0.0072
4    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 700 Training error: 0.0053 Testing error: 0.0086

Window size: 2
------------------------------------------------------------
5    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 500 Training error: 0.0041 Testing error: 0.0138
6    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 700 Training error: 0.0036 Testing error: 0.0119
7    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 500 Training error: 0.0047 Testing error: 0.0154
8    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 700 Training error: 0.0041 Testing error: 0.0072

Window size: 3
------------------------

In [1]:
ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)

NameError: name 'ModelLoader' is not defined

In [ ]:
window_sizes = [2, 5, 6, 7]
dropouts =  [0.25, 0.4]
learn_rates = [0.001]
epochs = [500,1000,1500]
batch_size = 30

for i in range(len(all_combination)-1):
    print(all_combination[i])
    print("{0} {1} {2}".format("-"*30, i + 1, "-"*30))
    result = model_selector(symbols[2], window_sizes, learn_rates, dropouts, epochs, batch_size,target_length=1,all_day=False,verbose=1,column=all_combination[i])

    print("\nResults : ")
    print("-"*60)
    print(result[0])

    print(result[1])
    ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)

In [25]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

[]

In [19]:
result = model_selector(symbols[2], window_sizes, learn_rates, dropouts, epochs, batch_size,verbose=1)

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
ModelLoader.save(result[1]['ticker'],result[0],result[1])
print("Saved trained model for {}".format(result[1]['ticker']))

*** Best Model Selection for AAPL ***

Window size: 3
------------------------------------------------------------
1    > Learn rate: 0.0100 Dropout: 0.25 Epoch: 300 Training error: 0.0032 Testing error: 0.0095
2    > Learn rate: 0.0100 Dropout: 0.25 Epoch: 500 Training error: 0.0032 Testing error: 0.0106
3    > Learn rate: 0.0100 Dropout: 0.40 Epoch: 300 Training error: 0.0044 Testing error: 0.0112
4    > Learn rate: 0.0100 Dropout: 0.40 Epoch: 500 Training error: 0.0042 Testing error: 0.0174
5    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 300 Training error: 0.0035 Testing error: 0.0090
6    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 500 Training error: 0.0035 Testing error: 0.0077
7    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 300 Training error: 0.0044 Testing error: 0.0093
8    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 500 Training error: 0.0042 Testing error: 0.0082

Window size: 5
------------------------------------------------------------
9    > Learn rate: 0.0100 Dropout: 0.25 

In [8]:
from keras import backend as K
import tensorflow as tf

for ticker in symbols[1:]:
    #release memory
    K.clear_session()
    tf.reset_default_graph()
    
    result = model_selector(ticker, window_sizes, learn_rates, dropouts, epochs, batch_size,verbose=2)
    
    #save trained model
    ModelLoader.save(result[1]['ticker'],result[0],result[1])
    print(" ==> Saved trained model for {}".format(result[1]['ticker']))


Model selection summary for AXP with window size of 10:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0035 Testing error: 0.0053
 ==> Saved trained model for AXP

Model selection summary for AAPL with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0031 Testing error: 0.0109
 ==> Saved trained model for AAPL

Model selection summary for BA with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0059 Testing error: 0.0031
 ==> Saved trained model for BA

Model selection summary for CAT with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0010 Dropout: 0.25 Epoch: 200 Training error: 0.0030 Testing error: 0.0055
 ==> Saved trained model for CAT

Model selection summar